In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [156]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

path = "../../../../"
filename = "Datafiles/bostonmarathon/results/2013/results.csv"
print("fullfilename = {}".format(path+filename))
df = pd.read_csv(path+filename)
print("There are {} rows in this file.".format(df.shape[0]))

fullfilename = ../../../../Datafiles/bostonmarathon/results/2013/results.csv
There are 16164 rows in this file.


In [154]:
df.dtypes

25k           float64
age             int64
name           object
division        int64
10k           float64
gender         object
half          float64
official      float64
bib            object
ctz            object
country        object
overall         int64
pace          float64
state          object
30k           float64
5k            float64
genderdiv       int64
20k           float64
35k           float64
city           object
40k           float64
gender_int      int64
bib_int         int64
dtype: object

In [204]:
df['gender_int'] = np.where(df['gender'] == 'M', 1, 0)
df['bib_int'] = df['bib'].replace(to_replace=r'[W|F]', value='-', regex=True).astype(int)
# element_wise_std = df2.apply(lambda x: np.std(np.array(x), 0))

# df['bib_num'] = df['bib'].replace({r'W.*'}, {r'\-.*})
kcolumns = ['5k', '10k', '20k', '25k', '30k', '35k', '40k', 'half']
for kcol in kcolumns:
    df[kcol] = np.where(df[kcol] == '-', 0, df[kcol])
    df[kcol] = df[kcol].astype(float)
    
# df['RaceStdDev'] = df.apply(lambda x: np.std(df.overall/26.2,df.age) )
# 'official' --> what is official???
# removing bib_int, as it does not really give us any info
df['5kpace']   = df['5k']/5.0
df['10kpace']  = df['10k']/10.0
df['20kpace']  = df['20k']/20.0
df['halfpace'] = df['half']/21.095
df['25kpace']  = df['25k']/25.0
df['30kpace']  = df['30k']/30.0
df['35kpace']  = df['35k']/35.0
df['40kpace']  = df['40k']/40.0
df['officialpace'] = df['official']/42.19
df['raceavg'] = df[['5kpace','10kpace','20kpace','halfpace','25kpace','30kpace','35kpace','40kpace','overallpace']].mean()
df['racestd'] = df[['5kpace','10kpace','20kpace','halfpace','25kpace','30kpace','35kpace','40kpace','overallpace']].std()

X = df[['age', 'division', 'gender_int','genderdiv', 'country', 
         '5k', '10k', '20k', '25k','30k','35k', 'half','40k']]
X = pd.get_dummies(X)
# X_norm = normalize(X)
y = df['overall']



In [206]:
# display(X.columns)
# display(df.columns)
display(df.head(20))
display(df.dtypes)
# display(df.describe())
# display(df.head(20))
# df['bib_int'] = df['bib'].replace(to_replace=r'[W|F]', value='-', regex=True).astype(int)
# df['bib_int'] = df['bib_int'].astype(int)
# display(X.head)
print("there are {} columns in this data frame.".format(X.shape[1]))

,25k,age,name,division,10k,gender,half,official,bib,ctz,...,20kpace,halfpace,25kpace,30kpace,35kpace,40kpace,overallpace,raceavg,racestd,officialpace
0,49.87,28,"Cassidy, Josh R.",9,18.18,M,40.93,90.90,W1,NaN,...,1.9400,1.940270,1.9948,2.069000,2.135143,2.13875,0.213321,NaN,NaN,2.154539
1,77.27,30,"Korir, Wesley",5,30.90,M,64.90,132.50,1,NaN,...,3.0760,3.076558,3.0908,3.099000,3.108000,3.11925,0.118511,NaN,NaN,3.140555
2,77.23,23,"Desisa, Lelisa",1,30.90,M,64.92,130.37,2,NaN,...,3.0765,3.077507,3.0892,3.090667,3.105143,3.09450,0.023702,NaN,NaN,3.090069
3,50.50,32,"Fearnley, Kurt H.",5,18.73,M,42.00,88.43,W2,NaN,...,1.9940,1.990993,2.0200,2.045000,2.085714,2.08575,0.118511,NaN,NaN,2.095994
4,48.75,39,"Hokinoue, Kota",3,18.18,M,40.57,87.22,W3,NaN,...,1.9275,1.923205,1.9500,1.997333,2.048000,2.04700,0.071107,NaN,NaN,2.067315
5,77.25,28,"Gebremariam, Gebregziabher",3,30.88,M,64.92,130.47,3,NaN,...,3.0765,3.077507,3.0900,3.090667,3.105714,3.09450,0.071107,NaN,NaN,3.092439
6,50.50,42,"Soejima, Masazumi",6,18.73,M,41.98,90.02,W4,NaN,...,1.9935,1.990045,2.0200,2.054333,2.114286,2.11675,0.142214,NaN,NaN,2.133681
7,77.23,28,"Geneti, Markos",6,30.88,M,64.90,132.73,4,NaN,...,3.0760,3.076558,3.0892,3.097667,3.116286,3.13000,0.142214,NaN,NaN,3.146006
8,52.62,49,"Schabort, Krige",11,19.77,M,43.97,91.78,W5,NaN,...,2.0885,2.084380,2.1048,2.133333,2.170000,2.16200,0.260725,NaN,NaN,2.175397
9,77.25,32,"Merga, Deriba",25,30.90,M,64.92,141.67,5,NaN,...,3.0765,3.077507,3.0900,3.092333,3.175143,3.29950,0.639962,NaN,NaN,3.357905


25k             float64
age               int64
name             object
division          int64
10k             float64
gender           object
half            float64
official        float64
bib              object
ctz              object
country          object
overall           int64
pace            float64
state            object
30k             float64
5k              float64
genderdiv         int64
20k             float64
35k             float64
city             object
40k             float64
gender_int        int64
bib_int           int64
RaceStdDev      float64
5kpace          float64
10kpace         float64
20kpace         float64
halfpace        float64
25kpace         float64
30kpace         float64
35kpace         float64
40kpace         float64
overallpace     float64
raceavg         float64
racestd         float64
officialpace    float64
dtype: object

there are 80 columns in this data frame.


Ignoring the specific colors assigned to each cluster, it appears that the two-cluster solution yields the most consistent results in terms of cluster center and cluster shape. Don't forget, however, that we are evaluating this based on two features from PCA rather than the n-dimensional data actually used by $k$-means, so the information value of these plots depends on how much information is retained by the PCA.

In [203]:
# # Get predicted clusters.
# full_pred = KMeans(n_clusters=2, random_state=42).fit_predict(X_norm)

# pd.crosstab(y, full_pred) 

In [141]:
from sklearn import metrics
    
metrics.adjusted_rand_score(y, full_pred)

1.2279077284978328e-06

This tells us that our $k$-means clustering solution is close to random.  Uh oh...  Perhaps a different algorithm would do better.  $ARI$ can be used effectively on all clustering algorithm types, so it is great for comparing different algorithm solutions to one another.

# Similarity: silhouette coefficient

What if we don't have a ground truth?  Then we need to use other criteria to evaluate clusters, such as similarity.  A good cluster is made up of datapoints that are more similar to one another than they are to datapoints in other clusters.  One way of computing similarity is the silhouette coefficient.

For each datapoint, the _silhouette coefficient_ is the difference between the mean distance between that datapoint and all other points in its cluster ($a_i$) and the mean distance between that datapoint and all other points in the *nearest other* cluster ($b_i$), divided by whichever of the two values is highest:  

$$\dfrac{b_i-a_i}{max(b_i,a_i)}$$

The mean of the silhouette coefficients for all datapoints is the silhouette coefficient for that clustering solution on that data.  Values range from -1 for very bad clusters to +1 for very dense clusters.  However, if clusters employ non-flat geometry, this index will not work well.

The silhouette coefficient is useful for situations where you want to compare the solutions of different clustering algorithms and you don't have a ground truth.  Silhouette coefficients can be compared across algorithms, creating a one-number index to represent a clustering solution and compare it against others.

Let's try the index on our heart disease data, using the two-cluster solution:

In [142]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances

for cluster in range(2,30):
    print("running with number of clusters = {}".format(cluster))
    for sample in [X]:
        model = KMeans(n_clusters=cluster, random_state=42).fit(sample)
        labels = model.labels_
        print(metrics.silhouette_score(sample, labels, metric='euclidean'))

running with number of clusters = 2
0.5247317819355591
running with number of clusters = 3
0.4696344897041349
running with number of clusters = 4
0.4274868846800941
running with number of clusters = 5
0.46776270558307226
running with number of clusters = 6
0.5023488229516737
running with number of clusters = 7
0.463675354208432
running with number of clusters = 8
0.4881700760073652
running with number of clusters = 9
0.4799860037558621
running with number of clusters = 10
0.45288340548398714
running with number of clusters = 11
0.4583591006135389
running with number of clusters = 12
0.471306985138108
running with number of clusters = 13
0.447171242830143
running with number of clusters = 14
0.44821905162310355
running with number of clusters = 15
0.4521288729258927
running with number of clusters = 16
0.4569703211753606
running with number of clusters = 17
0.4516190200977241
running with number of clusters = 18
0.4480721079368733
running with number of clusters = 19
0.44811080669919
ru

It appears that our two-cluster solution reliably produces clusters of datapoints that are closer to other datapoints in the cluster than they are to datapoints in the other clusters.  This is good.  What is also good is the consistency of the coefficient across our subsets.


# Thinking Like a Data Scientist: Defaults are Decisions too

As we continue our journey toward ever-more-complex models, the number of hyperparameters available in a model increases.  Each of these hyperparameters represents a choice about how the model will perform when dealing with your particular data.  In sklearn, as in most packages you will use as a data scientist, there are default settings for the hyperparameters.  If you're lucky, the defaults have been chosen because they are the most likely to perform decently for the most common types of data.  However, there's no guarantee that these settings will be best for your data.  

The thing to keep in mind is the choice to go with the default settings *is still a choice.*  For example, the Affinity Propagation algorithm can be run using only defaults (in fact, that's what we did in the last lesson):

```python
ap = AffinityPropagation()  
model = ap.fit(data)
```

Looking at the documentation reveals that we actually made a lot of choices just by going with the default.  A more informative look at our call would be:

```python
ap = AffinityPropagation(damping=0.5,
                         max_iter=200,
                         convergence_iter=15,
                         copy=True,
                         preference=None,
                         affinity='euclidean',
                         verbose=False)
```
               
We chose to use a damping parameter of .5, to stop after 200 iterations if the model hasn't converged yet, to require a model to run with no change for 15 iterations before accepting that we can converge, to make a copy of the input data (which uses extra memory), to tell it to treat all parameters as equally capable of becoming exemplars, to compute distances using affinity, and not to give us informative output during the fitting process.

These settings may or may not be best for your specific goals, so be sure to check them.  You may wish to explicitly declare all arguments each time you use a model, to make sure you have a record of the settings you used – especially as defaults may change with different versions of the same package!